In [52]:
import torch
import json
from tqdm import tqdm
from load import *

def compute_class_description_cosine_similarity(data):
    """
    Input:
    data: dictionary containing class descriptions
    
    Compute the cosine similarity between each class and all images,
    normalize these values, and save the results in JSON format.
    """
    device = torch.device(hparams['device'])
    model, preprocess = clip.load(hparams['model_size'], device=device, jit=False)
    model.eval()

    class_list = compute_class_list(data, sort_config=True)

    class_descriptor_dict = load_json(hparams['descriptor_fname'])
    class_list = compute_class_list(class_descriptor_dict, sort_config=True)
    descriptor_list = compute_descriptor_list(class_descriptor_dict, sort_config=True)

    class_list = [c.replace('-', ' ') for c in class_list]

    seed_everything(hparams['seed'])

    # Load the model and preprocessing
    print("Loading model...")
    device = torch.device(hparams['device'])
    model, preprocess = clip.load(hparams['model_size'], device=device, jit=False)
    model.eval()
    model.requires_grad_(False)

    # Encode descriptions and labels
    print("Encoding descriptions...")
    description_encodings = F.normalize(model.encode_text(clip.tokenize(descriptor_list).to(device)))
    label_encodings = F.normalize(model.encode_text(clip.tokenize(class_list).to(device)))

    cosine_similarity = torch.mm(description_encodings, label_encodings.T)
    print(cosine_similarity.size())

    cosine_similarity_per_class = cosine_similarity.cpu().detach().numpy()
    print("cosine_similarity:", cosine_similarity_per_class)

    # Calculate average cosine similarity for each class
    average_cosine_similarity = cosine_similarity.mean(dim=0).tolist()

    similarity = {}
    similarity[hparams['dataset']] = {}
    for i, class_name in enumerate(class_list):
        similarity[hparams['dataset']][class_name] = {}
        similarity[hparams['dataset']][class_name]["cosine_similarity_vector"] = cosine_similarity_per_class[i].tolist()
        if len(similarity[hparams['dataset']][class_name]["cosine_similarity_vector"]) == len(dataset_classes):
            similarity[hparams['dataset']][class_name]["cosine_similarity_vector_sorted"] = sorted(similarity[hparams['dataset']][class_name]["cosine_similarity_vector"], reverse=True)
            similarity[hparams['dataset']][class_name]["average_cosine_similarity"] = average_cosine_similarity[i]

    return similarity

In [53]:
descriptor_file_path = hparams['descriptor_fname']

class_descriptor_dict = load_json(descriptor_file_path)
analysis_dict = compute_class_description_cosine_similarity(class_descriptor_dict)

output_path_name = descriptor_file_path.split("/")[-1].split(".")[0].split("_")[-1]
json_output_path = f'class_analysis/class_analysis_{output_path_name}.json'
with open(json_output_path, 'w') as f:
    json.dump(analysis_dict, f, indent=4)

# print("Similarity:", cosine_similarity, f"\nAverage similarity: length {len(average_cosine_similiarity)}", average_cosine_similiarity)

Loading model...
Encoding descriptions...
torch.Size([563, 200])
cosine_similarity: [[0.5054 0.6377 0.4543 ... 0.6055 0.517  0.4739]
 [0.6753 0.8    0.624  ... 0.755  0.6177 0.6357]
 [0.5483 0.6973 0.498  ... 0.6724 0.528  0.503 ]
 ...
 [0.643  0.6357 0.5557 ... 0.711  0.6235 0.6406]
 [0.6646 0.7114 0.6284 ... 0.75   0.6426 0.6763]
 [0.614  0.6606 0.492  ... 0.6255 0.5176 0.5776]]


In [48]:
classwise_accuracies = {
    "091.Mockingbird": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": 53.33333536982536
    },
    "077.Tropical_Kingbird": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 46.666666865348816,
        "Difference": 40.00000059604645
    },
    "145.Elegant_Tern": {
        "Description-based Accuracy": 46.666666865348816,
        "CLIP-Standard Accuracy": 13.333334028720856,
        "Difference": 33.33333283662796
    },
    "092.Nighthawk": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 30.000001192092896
    },
    "005.Crested_Auklet": {
        "Description-based Accuracy": 50.0,
        "CLIP-Standard Accuracy": 21.42857164144516,
        "Difference": 28.57142835855484
    },
    "027.Shiny_Cowbird": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": 23.333334177732468
    },
    "128.Seaside_Sparrow": {
        "Description-based Accuracy": 40.00000059604645,
        "CLIP-Standard Accuracy": 16.66666716337204,
        "Difference": 23.333333432674408
    },
    "024.Red_faced_Cormorant": {
        "Description-based Accuracy": 68.18181872367859,
        "CLIP-Standard Accuracy": 45.45454680919647,
        "Difference": 22.727271914482117
    },
    "170.Mourning_Warbler": {
        "Description-based Accuracy": 26.66666805744171,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": 20.000001043081284
    },
    "018.Spotted_Catbird": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 13.333334028720856,
        "Difference": 20.000000298023224
    },
    "098.Scott_Oriole": {
        "Description-based Accuracy": 20.000000298023224,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 20.000000298023224
    },
    "070.Green_Violetear": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 63.333332538604736,
        "Difference": 19.999998807907104
    },
    "017.Cardinal": {
        "Description-based Accuracy": 74.0740716457367,
        "CLIP-Standard Accuracy": 55.55555820465088,
        "Difference": 18.518513441085815
    },
    "007.Parakeet_Auklet": {
        "Description-based Accuracy": 21.739129722118378,
        "CLIP-Standard Accuracy": 4.3478261679410934,
        "Difference": 17.391303554177284
    },
    "064.Ring_billed_Gull": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": 16.66666865348816
    },
    "183.Northern_Waterthrush": {
        "Description-based Accuracy": 66.66666865348816,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": 16.66666865348816
    },
    "009.Brewer_Blackbird": {
        "Description-based Accuracy": 17.241379618644714,
        "CLIP-Standard Accuracy": 3.448275849223137,
        "Difference": 13.793103769421577
    },
    "116.Chipping_Sparrow": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 46.666666865348816,
        "Difference": 13.333335518836975
    },
    "154.Red_eyed_Vireo": {
        "Description-based Accuracy": 53.33333611488342,
        "CLIP-Standard Accuracy": 40.00000059604645,
        "Difference": 13.333335518836975
    },
    "118.House_Sparrow": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 16.66666716337204,
        "Difference": 13.333334028720856
    },
    "121.Grasshopper_Sparrow": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 20.000000298023224,
        "Difference": 13.333334028720856
    },
    "083.White_breasted_Kingfisher": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 60.00000238418579,
        "Difference": 13.333332538604736
    },
    "112.Great_Grey_Shrike": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 66.66666865348816,
        "Difference": 13.333332538604736
    },
    "131.Vesper_Sparrow": {
        "Description-based Accuracy": 50.0,
        "CLIP-Standard Accuracy": 36.666667461395264,
        "Difference": 13.333332538604736
    },
    "190.Red_cockaded_Woodpecker": {
        "Description-based Accuracy": 10.34482792019844,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 10.34482792019844
    },
    "023.Brandt_Cormorant": {
        "Description-based Accuracy": 27.586206793785095,
        "CLIP-Standard Accuracy": 17.241379618644714,
        "Difference": 10.34482717514038
    },
    "045.Northern_Fulmar": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": 10.000002384185791
    },
    "063.Ivory_Gull": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 69.9999988079071,
        "Difference": 10.000002384185791
    },
    "073.Blue_Jay": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": 10.000002384185791
    },
    "062.Herring_Gull": {
        "Description-based Accuracy": 26.66666805744171,
        "CLIP-Standard Accuracy": 16.66666716337204,
        "Difference": 10.000000894069672
    },
    "142.Black_Tern": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 33.33333432674408,
        "Difference": 9.999999403953552
    },
    "173.Orange_crowned_Warbler": {
        "Description-based Accuracy": 23.333333432674408,
        "CLIP-Standard Accuracy": 13.333334028720856,
        "Difference": 9.999999403953552
    },
    "104.American_Pipit": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 9.999996423721313
    },
    "108.White_necked_Raven": {
        "Description-based Accuracy": 76.66666507720947,
        "CLIP-Standard Accuracy": 66.66666865348816,
        "Difference": 9.999996423721313
    },
    "182.Yellow_Warbler": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 9.999996423721313
    },
    "197.Marsh_Wren": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 73.33333492279053,
        "Difference": 9.999996423721313
    },
    "084.Red_legged_Kittiwake": {
        "Description-based Accuracy": 82.6086938381195,
        "CLIP-Standard Accuracy": 73.9130437374115,
        "Difference": 8.695650100708008
    },
    "015.Lazuli_Bunting": {
        "Description-based Accuracy": 85.71428656578064,
        "CLIP-Standard Accuracy": 78.57142686843872,
        "Difference": 7.142859697341919
    },
    "166.Golden_winged_Warbler": {
        "Description-based Accuracy": 79.31034564971924,
        "CLIP-Standard Accuracy": 72.41379022598267,
        "Difference": 6.896555423736572
    },
    "141.Artic_Tern": {
        "Description-based Accuracy": 20.68965584039688,
        "CLIP-Standard Accuracy": 13.793103396892548,
        "Difference": 6.8965524435043335
    },
    "039.Least_Flycatcher": {
        "Description-based Accuracy": 10.34482792019844,
        "CLIP-Standard Accuracy": 3.448275849223137,
        "Difference": 6.896552070975304
    },
    "158.Bay_breasted_Warbler": {
        "Description-based Accuracy": 6.666667014360428,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 6.666667014360428
    },
    "135.Bank_Swallow": {
        "Description-based Accuracy": 10.000000149011612,
        "CLIP-Standard Accuracy": 3.333333507180214,
        "Difference": 6.666666641831398
    },
    "038.Great_Crested_Flycatcher": {
        "Description-based Accuracy": 20.000000298023224,
        "CLIP-Standard Accuracy": 13.333334028720856,
        "Difference": 6.666666269302368
    },
    "106.Horned_Puffin": {
        "Description-based Accuracy": 93.33333373069763,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": 6.666666269302368
    },
    "109.American_Redstart": {
        "Description-based Accuracy": 69.9999988079071,
        "CLIP-Standard Accuracy": 63.333332538604736,
        "Difference": 6.666666269302368
    },
    "140.Summer_Tanager": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 73.33333492279053,
        "Difference": 6.666666269302368
    },
    "175.Pine_Warbler": {
        "Description-based Accuracy": 50.0,
        "CLIP-Standard Accuracy": 43.33333373069763,
        "Difference": 6.666666269302368
    },
    "176.Prairie_Warbler": {
        "Description-based Accuracy": 63.333332538604736,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": 6.666666269302368
    },
    "177.Prothonotary_Warbler": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": 6.666666269302368
    },
    "192.Downy_Woodpecker": {
        "Description-based Accuracy": 50.0,
        "CLIP-Standard Accuracy": 43.33333373069763,
        "Difference": 6.666666269302368
    },
    "199.Winter_Wren": {
        "Description-based Accuracy": 56.66666626930237,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": 6.666666269302368
    },
    "032.Mangrove_Cuckoo": {
        "Description-based Accuracy": 56.521737575531006,
        "CLIP-Standard Accuracy": 52.173912525177,
        "Difference": 4.347825050354004
    },
    "178.Swainson_Warbler": {
        "Description-based Accuracy": 42.307692766189575,
        "CLIP-Standard Accuracy": 38.46153914928436,
        "Difference": 3.8461536169052124
    },
    "028.Brown_Creeper": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 96.55172228813171,
        "Difference": 3.448277711868286
    },
    "119.Field_Sparrow": {
        "Description-based Accuracy": 58.62069129943848,
        "CLIP-Standard Accuracy": 55.17241358757019,
        "Difference": 3.448277711868286
    },
    "149.Brown_Thrasher": {
        "Description-based Accuracy": 93.1034505367279,
        "CLIP-Standard Accuracy": 89.65517282485962,
        "Difference": 3.448277711868286
    },
    "004.Groove_billed_Ani": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": 3.333336114883423
    },
    "031.Black_billed_Cuckoo": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": 3.333336114883423
    },
    "036.Northern_Flicker": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 3.333336114883423
    },
    "051.Horned_Grebe": {
        "Description-based Accuracy": 66.66666865348816,
        "CLIP-Standard Accuracy": 63.333332538604736,
        "Difference": 3.333336114883423
    },
    "054.Blue_Grosbeak": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": 3.333336114883423
    },
    "056.Pine_Grosbeak": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 3.333336114883423
    },
    "076.Dark_eyed_Junco": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 3.333336114883423
    },
    "100.Brown_Pelican": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 3.333336114883423
    },
    "133.White_throated_Sparrow": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": 3.333336114883423
    },
    "159.Black_and_white_Warbler": {
        "Description-based Accuracy": 93.33333373069763,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": 3.333336114883423
    },
    "174.Palm_Warbler": {
        "Description-based Accuracy": 53.33333611488342,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": 3.333336114883423
    },
    "059.California_Gull": {
        "Description-based Accuracy": 26.66666805744171,
        "CLIP-Standard Accuracy": 23.333333432674408,
        "Difference": 3.3333346247673035
    },
    "071.Long_tailed_Jaeger": {
        "Description-based Accuracy": 26.66666805744171,
        "CLIP-Standard Accuracy": 23.333333432674408,
        "Difference": 3.3333346247673035
    },
    "126.Nelson_Sharp_tailed_Sparrow": {
        "Description-based Accuracy": 3.333333507180214,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 3.333333507180214
    },
    "029.American_Crow": {
        "Description-based Accuracy": 36.666667461395264,
        "CLIP-Standard Accuracy": 33.33333432674408,
        "Difference": 3.333333134651184
    },
    "067.Anna_Hummingbird": {
        "Description-based Accuracy": 23.333333432674408,
        "CLIP-Standard Accuracy": 20.000000298023224,
        "Difference": 3.333333134651184
    },
    "074.Florida_Jay": {
        "Description-based Accuracy": 50.0,
        "CLIP-Standard Accuracy": 46.666666865348816,
        "Difference": 3.333333134651184
    },
    "102.Western_Wood_Pewee": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 30.000001192092896,
        "Difference": 3.333333134651184
    },
    "123.Henslow_Sparrow": {
        "Description-based Accuracy": 40.00000059604645,
        "CLIP-Standard Accuracy": 36.666667461395264,
        "Difference": 3.333333134651184
    },
    "055.Evening_Grosbeak": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": 3.3333301544189453
    },
    "090.Red_breasted_Merganser": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 3.3333301544189453
    },
    "132.White_crowned_Sparrow": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 3.3333301544189453
    },
    "139.Scarlet_Tanager": {
        "Description-based Accuracy": 76.66666507720947,
        "CLIP-Standard Accuracy": 73.33333492279053,
        "Difference": 3.3333301544189453
    },
    "198.Rock_Wren": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 3.3333301544189453
    },
    "012.Yellow_headed_Blackbird": {
        "Description-based Accuracy": 96.15384340286255,
        "CLIP-Standard Accuracy": 96.15384340286255,
        "Difference": 0.0
    },
    "016.Painted_Bunting": {
        "Description-based Accuracy": 92.85714030265808,
        "CLIP-Standard Accuracy": 92.85714030265808,
        "Difference": 0.0
    },
    "022.Chuck_will_Widow": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "030.Fish_Crow": {
        "Description-based Accuracy": 6.666667014360428,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": 0.0
    },
    "037.Acadian_Flycatcher": {
        "Description-based Accuracy": 65.51724076271057,
        "CLIP-Standard Accuracy": 65.51724076271057,
        "Difference": 0.0
    },
    "042.Vermilion_Flycatcher": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 100.0,
        "Difference": 0.0
    },
    "048.European_Goldfinch": {
        "Description-based Accuracy": 96.66666388511658,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 0.0
    },
    "053.Western_Grebe": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 100.0,
        "Difference": 0.0
    },
    "057.Rose_breasted_Grosbeak": {
        "Description-based Accuracy": 96.66666388511658,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 0.0
    },
    "058.Pigeon_Guillemot": {
        "Description-based Accuracy": 71.42857313156128,
        "CLIP-Standard Accuracy": 71.42857313156128,
        "Difference": 0.0
    },
    "061.Heermann_Gull": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "065.Slaty_backed_Gull": {
        "Description-based Accuracy": 44.999998807907104,
        "CLIP-Standard Accuracy": 44.999998807907104,
        "Difference": 0.0
    },
    "072.Pomarine_Jaeger": {
        "Description-based Accuracy": 6.666667014360428,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": 0.0
    },
    "079.Belted_Kingfisher": {
        "Description-based Accuracy": 96.66666388511658,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 0.0
    },
    "080.Green_Kingfisher": {
        "Description-based Accuracy": 16.66666716337204,
        "CLIP-Standard Accuracy": 16.66666716337204,
        "Difference": 0.0
    },
    "081.Pied_Kingfisher": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": 0.0
    },
    "082.Ringed_Kingfisher": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 43.33333373069763,
        "Difference": 0.0
    },
    "085.Horned_Lark": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": 0.0
    },
    "087.Mallard": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 0.0
    },
    "088.Western_Meadowlark": {
        "Description-based Accuracy": 96.66666388511658,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": 0.0
    },
    "089.Hooded_Merganser": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 0.0
    },
    "096.Hooded_Oriole": {
        "Description-based Accuracy": 40.00000059604645,
        "CLIP-Standard Accuracy": 40.00000059604645,
        "Difference": 0.0
    },
    "101.White_Pelican": {
        "Description-based Accuracy": 100.0,
        "CLIP-Standard Accuracy": 100.0,
        "Difference": 0.0
    },
    "103.Sayornis": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "105.Whip_poor_Will": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "110.Geococcyx": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "113.Baird_Sparrow": {
        "Description-based Accuracy": 5.000000074505806,
        "CLIP-Standard Accuracy": 5.000000074505806,
        "Difference": 0.0
    },
    "114.Black_throated_Sparrow": {
        "Description-based Accuracy": 10.000000149011612,
        "CLIP-Standard Accuracy": 10.000000149011612,
        "Difference": 0.0
    },
    "120.Fox_Sparrow": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 33.33333432674408,
        "Difference": 0.0
    },
    "122.Harris_Sparrow": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 0.0,
        "Difference": 0.0
    },
    "124.Le_Conte_Sparrow": {
        "Description-based Accuracy": 41.37931168079376,
        "CLIP-Standard Accuracy": 41.37931168079376,
        "Difference": 0.0
    },
    "130.Tree_Sparrow": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 30.000001192092896,
        "Difference": 0.0
    },
    "148.Green_tailed_Towhee": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": 0.0
    },
    "152.Blue_headed_Vireo": {
        "Description-based Accuracy": 40.00000059604645,
        "CLIP-Standard Accuracy": 40.00000059604645,
        "Difference": 0.0
    },
    "156.White_eyed_Vireo": {
        "Description-based Accuracy": 10.000000149011612,
        "CLIP-Standard Accuracy": 10.000000149011612,
        "Difference": 0.0
    },
    "164.Cerulean_Warbler": {
        "Description-based Accuracy": 66.66666865348816,
        "CLIP-Standard Accuracy": 66.66666865348816,
        "Difference": 0.0
    },
    "165.Chestnut_sided_Warbler": {
        "Description-based Accuracy": 56.66666626930237,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": 0.0
    },
    "167.Hooded_Warbler": {
        "Description-based Accuracy": 23.333333432674408,
        "CLIP-Standard Accuracy": 23.333333432674408,
        "Difference": 0.0
    },
    "168.Kentucky_Warbler": {
        "Description-based Accuracy": 10.34482792019844,
        "CLIP-Standard Accuracy": 10.34482792019844,
        "Difference": 0.0
    },
    "180.Wilson_Warbler": {
        "Description-based Accuracy": 3.333333507180214,
        "CLIP-Standard Accuracy": 3.333333507180214,
        "Difference": 0.0
    },
    "185.Bohemian_Waxwing": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 0.0
    },
    "186.Cedar_Waxwing": {
        "Description-based Accuracy": 93.33333373069763,
        "CLIP-Standard Accuracy": 93.33333373069763,
        "Difference": 0.0
    },
    "191.Red_headed_Woodpecker": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": 0.0
    },
    "193.Bewick_Wren": {
        "Description-based Accuracy": 3.333333507180214,
        "CLIP-Standard Accuracy": 3.333333507180214,
        "Difference": 0.0
    },
    "021.Eastern_Towhee": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": -3.3333301544189453
    },
    "046.Gadwall": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": -3.3333301544189453
    },
    "069.Rufous_Hummingbird": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": -3.3333301544189453
    },
    "094.White_breasted_Nuthatch": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": -3.3333301544189453
    },
    "099.Ovenbird": {
        "Description-based Accuracy": 66.66666865348816,
        "CLIP-Standard Accuracy": 69.9999988079071,
        "Difference": -3.3333301544189453
    },
    "136.Barn_Swallow": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": -3.3333301544189453
    },
    "194.Cactus_Wren": {
        "Description-based Accuracy": 93.33333373069763,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": -3.3333301544189453
    },
    "041.Scissor_tailed_Flycatcher": {
        "Description-based Accuracy": 46.666666865348816,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -3.333333134651184
    },
    "050.Eared_Grebe": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 33.33333432674408,
        "Difference": -3.333333134651184
    },
    "129.Song_Sparrow": {
        "Description-based Accuracy": 46.666666865348816,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -3.333333134651184
    },
    "155.Warbling_Vireo": {
        "Description-based Accuracy": 20.000000298023224,
        "CLIP-Standard Accuracy": 23.333333432674408,
        "Difference": -3.333333134651184
    },
    "163.Cape_May_Warbler": {
        "Description-based Accuracy": 6.666667014360428,
        "CLIP-Standard Accuracy": 10.000000149011612,
        "Difference": -3.333333134651184
    },
    "171.Myrtle_Warbler": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 46.666666865348816,
        "Difference": -3.333333134651184
    },
    "146.Forsters_Tern": {
        "Description-based Accuracy": 3.333333507180214,
        "CLIP-Standard Accuracy": 6.666667014360428,
        "Difference": -3.333333507180214
    },
    "001.Black_footed_Albatross": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -3.333336114883423
    },
    "010.Red_winged_Blackbird": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -3.333336114883423
    },
    "086.Pacific_Loon": {
        "Description-based Accuracy": 69.9999988079071,
        "CLIP-Standard Accuracy": 73.33333492279053,
        "Difference": -3.333336114883423
    },
    "093.Clark_Nutcracker": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -3.333336114883423
    },
    "134.Cape_Glossy_Starling": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 93.33333373069763,
        "Difference": -3.333336114883423
    },
    "150.Sage_Thrasher": {
        "Description-based Accuracy": 76.66666507720947,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": -3.333336114883423
    },
    "196.House_Wren": {
        "Description-based Accuracy": 56.66666626930237,
        "CLIP-Standard Accuracy": 60.00000238418579,
        "Difference": -3.333336114883423
    },
    "200.Common_Yellowthroat": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -3.333336114883423
    },
    "078.Gray_Kingbird": {
        "Description-based Accuracy": 68.96551847457886,
        "CLIP-Standard Accuracy": 72.41379022598267,
        "Difference": -3.4482717514038086
    },
    "169.Magnolia_Warbler": {
        "Description-based Accuracy": 68.96551847457886,
        "CLIP-Standard Accuracy": 72.41379022598267,
        "Difference": -3.4482717514038086
    },
    "060.Glaucous_winged_Gull": {
        "Description-based Accuracy": 20.68965584039688,
        "CLIP-Standard Accuracy": 24.13793057203293,
        "Difference": -3.4482747316360474
    },
    "097.Orchard_Oriole": {
        "Description-based Accuracy": 34.48275923728943,
        "CLIP-Standard Accuracy": 37.931033968925476,
        "Difference": -3.4482747316360474
    },
    "117.Clay_colored_Sparrow": {
        "Description-based Accuracy": 3.448275849223137,
        "CLIP-Standard Accuracy": 6.896551698446274,
        "Difference": -3.448275849223137
    },
    "153.Philadelphia_Vireo": {
        "Description-based Accuracy": 3.448275849223137,
        "CLIP-Standard Accuracy": 6.896551698446274,
        "Difference": -3.448275849223137
    },
    "043.Yellow_bellied_Flycatcher": {
        "Description-based Accuracy": 6.896551698446274,
        "CLIP-Standard Accuracy": 10.34482792019844,
        "Difference": -3.4482762217521667
    },
    "125.Lincoln_Sparrow": {
        "Description-based Accuracy": 6.896551698446274,
        "CLIP-Standard Accuracy": 10.34482792019844,
        "Difference": -3.4482762217521667
    },
    "019.Gray_Catbird": {
        "Description-based Accuracy": 75.86206793785095,
        "CLIP-Standard Accuracy": 79.31034564971924,
        "Difference": -3.448277711868286
    },
    "160.Black_throated_Blue_Warbler": {
        "Description-based Accuracy": 79.31034564971924,
        "CLIP-Standard Accuracy": 82.75862336158752,
        "Difference": -3.448277711868286
    },
    "075.Green_Jay": {
        "Description-based Accuracy": 88.88888955116272,
        "CLIP-Standard Accuracy": 92.59259104728699,
        "Difference": -3.7037014961242676
    },
    "013.Bobolink": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -6.666666269302368
    },
    "014.Indigo_Bunting": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": -6.666666269302368
    },
    "035.Purple_Finch": {
        "Description-based Accuracy": 86.66666746139526,
        "CLIP-Standard Accuracy": 93.33333373069763,
        "Difference": -6.666666269302368
    },
    "052.Pied_billed_Grebe": {
        "Description-based Accuracy": 80.0000011920929,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -6.666666269302368
    },
    "107.Common_Raven": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -6.666666269302368
    },
    "137.Cliff_Swallow": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -6.666666269302368
    },
    "144.Common_Tern": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 40.00000059604645,
        "Difference": -6.666666269302368
    },
    "162.Canada_Warbler": {
        "Description-based Accuracy": 43.33333373069763,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -6.666666269302368
    },
    "172.Nashville_Warbler": {
        "Description-based Accuracy": 56.66666626930237,
        "CLIP-Standard Accuracy": 63.333332538604736,
        "Difference": -6.666666269302368
    },
    "188.Pileated_Woodpecker": {
        "Description-based Accuracy": 89.99999761581421,
        "CLIP-Standard Accuracy": 96.66666388511658,
        "Difference": -6.666666269302368
    },
    "189.Red_bellied_Woodpecker": {
        "Description-based Accuracy": 83.33333134651184,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": -6.666666269302368
    },
    "181.Worm_eating_Warbler": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 6.896551698446274,
        "Difference": -6.896551698446274
    },
    "003.Sooty_Albatross": {
        "Description-based Accuracy": 25.0,
        "CLIP-Standard Accuracy": 32.14285671710968,
        "Difference": -7.14285671710968
    },
    "151.Black_capped_Vireo": {
        "Description-based Accuracy": 61.90476417541504,
        "CLIP-Standard Accuracy": 71.42857313156128,
        "Difference": -9.52380895614624
    },
    "049.Boat_tailed_Grackle": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": -9.999996423721313
    },
    "187.American_Three_toed_Woodpecker": {
        "Description-based Accuracy": 85.00000238418579,
        "CLIP-Standard Accuracy": 94.9999988079071,
        "Difference": -9.999996423721313
    },
    "011.Rusty_Blackbird": {
        "Description-based Accuracy": 46.666666865348816,
        "CLIP-Standard Accuracy": 56.66666626930237,
        "Difference": -9.999999403953552
    },
    "040.Olive_sided_Flycatcher": {
        "Description-based Accuracy": 26.66666805744171,
        "CLIP-Standard Accuracy": 36.666667461395264,
        "Difference": -9.999999403953552
    },
    "127.Savannah_Sparrow": {
        "Description-based Accuracy": 33.33333432674408,
        "CLIP-Standard Accuracy": 43.33333373069763,
        "Difference": -9.999999403953552
    },
    "066.Western_Gull": {
        "Description-based Accuracy": 0.0,
        "CLIP-Standard Accuracy": 10.000000149011612,
        "Difference": -10.000000149011612
    },
    "161.Blue_winged_Warbler": {
        "Description-based Accuracy": 3.333333507180214,
        "CLIP-Standard Accuracy": 13.333334028720856,
        "Difference": -10.000000521540642
    },
    "184.Louisiana_Waterthrush": {
        "Description-based Accuracy": 16.66666716337204,
        "CLIP-Standard Accuracy": 26.66666805744171,
        "Difference": -10.000000894069672
    },
    "068.Ruby_throated_Hummingbird": {
        "Description-based Accuracy": 69.9999988079071,
        "CLIP-Standard Accuracy": 80.0000011920929,
        "Difference": -10.000002384185791
    },
    "138.Tree_Swallow": {
        "Description-based Accuracy": 76.66666507720947,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -10.000002384185791
    },
    "020.Yellow_breasted_Chat": {
        "Description-based Accuracy": 13.793103396892548,
        "CLIP-Standard Accuracy": 24.13793057203293,
        "Difference": -10.34482717514038
    },
    "115.Brewer_Sparrow": {
        "Description-based Accuracy": 13.793103396892548,
        "CLIP-Standard Accuracy": 24.13793057203293,
        "Difference": -10.34482717514038
    },
    "034.Gray_crowned_Rosy_Finch": {
        "Description-based Accuracy": 24.13793057203293,
        "CLIP-Standard Accuracy": 34.48275923728943,
        "Difference": -10.3448286652565
    },
    "002.Laysan_Albatross": {
        "Description-based Accuracy": 60.00000238418579,
        "CLIP-Standard Accuracy": 73.33333492279053,
        "Difference": -13.333332538604736
    },
    "044.Frigatebird": {
        "Description-based Accuracy": 76.66666507720947,
        "CLIP-Standard Accuracy": 89.99999761581421,
        "Difference": -13.333332538604736
    },
    "047.American_Goldfinch": {
        "Description-based Accuracy": 73.33333492279053,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -13.333332538604736
    },
    "195.Carolina_Wren": {
        "Description-based Accuracy": 63.333332538604736,
        "CLIP-Standard Accuracy": 76.66666507720947,
        "Difference": -13.333332538604736
    },
    "033.Yellow_billed_Cuckoo": {
        "Description-based Accuracy": 34.48275923728943,
        "CLIP-Standard Accuracy": 48.27586114406586,
        "Difference": -13.793101906776428
    },
    "157.Yellow_throated_Vireo": {
        "Description-based Accuracy": 34.48275923728943,
        "CLIP-Standard Accuracy": 48.27586114406586,
        "Difference": -13.793101906776428
    },
    "147.Least_Tern": {
        "Description-based Accuracy": 66.66666865348816,
        "CLIP-Standard Accuracy": 83.33333134651184,
        "Difference": -16.66666269302368
    },
    "179.Tennessee_Warbler": {
        "Description-based Accuracy": 13.793103396892548,
        "CLIP-Standard Accuracy": 31.034481525421143,
        "Difference": -17.241378128528595
    },
    "025.Pelagic_Cormorant": {
        "Description-based Accuracy": 30.000001192092896,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -19.999998807907104
    },
    "111.Loggerhead_Shrike": {
        "Description-based Accuracy": 13.333334028720856,
        "CLIP-Standard Accuracy": 33.33333432674408,
        "Difference": -20.000000298023224
    },
    "008.Rhinoceros_Auklet": {
        "Description-based Accuracy": 27.77777910232544,
        "CLIP-Standard Accuracy": 50.0,
        "Difference": -22.22222089767456
    },
    "095.Baltimore_Oriole": {
        "Description-based Accuracy": 16.66666716337204,
        "CLIP-Standard Accuracy": 40.00000059604645,
        "Difference": -23.333333432674408
    },
    "026.Bronzed_Cowbird": {
        "Description-based Accuracy": 6.666667014360428,
        "CLIP-Standard Accuracy": 30.000001192092896,
        "Difference": -23.333334177732468
    },
    "006.Least_Auklet": {
        "Description-based Accuracy": 27.272728085517883,
        "CLIP-Standard Accuracy": 54.54545617103577,
        "Difference": -27.272728085517883
    },
    "143.Caspian_Tern": {
        "Description-based Accuracy": 56.66666626930237,
        "CLIP-Standard Accuracy": 86.66666746139526,
        "Difference": -30.000001192092896
    }
}

classes_and_differences = {}
for class_name in classwise_accuracies.keys():
    # print(class_name.split(".")[1:][0])
    class_name_edit = class_name.split(".")[1:][0].replace("_"," ")
    classes_and_differences[class_name_edit] = classwise_accuracies[class_name]["Difference"]

classes_and_differences = dict(sorted(classes_and_differences.items(), key=lambda item: item[1], reverse=True))
print(classes_and_differences)

{'Mockingbird': 53.33333536982536, 'Tropical Kingbird': 40.00000059604645, 'Elegant Tern': 33.33333283662796, 'Nighthawk': 30.000001192092896, 'Crested Auklet': 28.57142835855484, 'Shiny Cowbird': 23.333334177732468, 'Seaside Sparrow': 23.333333432674408, 'Red faced Cormorant': 22.727271914482117, 'Mourning Warbler': 20.000001043081284, 'Spotted Catbird': 20.000000298023224, 'Scott Oriole': 20.000000298023224, 'Green Violetear': 19.999998807907104, 'Cardinal': 18.518513441085815, 'Parakeet Auklet': 17.391303554177284, 'Ring billed Gull': 16.66666865348816, 'Northern Waterthrush': 16.66666865348816, 'Brewer Blackbird': 13.793103769421577, 'Chipping Sparrow': 13.333335518836975, 'Red eyed Vireo': 13.333335518836975, 'House Sparrow': 13.333334028720856, 'Grasshopper Sparrow': 13.333334028720856, 'White breasted Kingfisher': 13.333332538604736, 'Great Grey Shrike': 13.333332538604736, 'Vesper Sparrow': 13.333332538604736, 'Red cockaded Woodpecker': 10.34482792019844, 'Brandt Cormorant': 10